#### evaluate_llms.ipynb
#### Copyright (c) 2025, Joshua J Hamilton

In this notebook, we will evaluate the ability of different LLMs to tag classical music audio files.

I will evaluate some of the following LLMs:
* DeepSeek
* Claude
* Gemini
* GPT
* Llama
* Mistral

To evaluate the LLMs, I will
* Perform prompt engineering to generate prompts for each LLM. Validate prompts based on works I have tagged thus far.
* Evaluate prompts and LLMs and select one for testing.
* Test the optimized prompt on a test set containing the complete works of Beethoven, who was not in the validation set

Validation set:
| Genre     | Composer                 | Type of Work                    | Recording                                                   |
| --------- | ------------------------ | ------------------------------- | ----------------------------------------------------------- |
| Baroque   | Bach, Johann Sebastian   | Brandenburg Concertos           | Musica Antiqua Koln with Reinhard Goebel                    |
| Baroque   | Bach, Johann Sebastian   | Harpsichord Works               | Gustav Leonhardt                                            |
| Baroque   | Bach, Johann Sebastian   | Orchestral Suites               | Musica Antiqua Koln with Reinhard Goebel                    |
| Baroque   | Bach, Johann Sebastian   | Organ Works                     | Helmut Walcha                                               |
| Baroque   | Bach, Johann Sebastian   | Piano Works                     | Glenn Gould                                                 |
| Baroque   | Handel, George Frideric  | Concerti Grossi                 | Karl Richter                                                |
| Baroque   | Handel, George Frideric  | Keyboard Music                  | Scott Ross                                                  |
| Baroque   | Handel, George Frideric  | Organ Concertos                 | Karl Richter                                                |
| Baroque   | Handel, George Frideric  | Royal Fireworks, Water Music    | Academy of Ancient Music with Neville Marriner              |
| Baroque   | Vivaldi, Antonio         | Concerti                        | I Musici with Felix Ayo                                     |
| Baroque   | Vivaldi, Antonio         | The Four Seasons                | I Musici with Felix Ayo                                     |
| Classical | Haydn, Joseph            | Harpsichord and Organ Concertos | Ton Koopman                                                 |
| Classical | Haydn, Joseph            | Piano Sonatas                   | Ronald Brautigam                                            |
| Classical | Haydn, Joseph            | String Quartets                 | Auryn Quartet                                               |
| Classical | Haydn, Joseph            | Symphonies                      | Austro-Hungarian Haydn O with Adam Fischer                  |
| Classical | Mozart, Wolfgang Amadeus | Overtures                       | Staatskapelle Dresden with Colin Davis                      |
| Classical | Mozart, Wolfgang Amadeus | Piano Concertos                 | Andras Schiff                                               |
| Classical | Mozart, Wolfgang Amadeus | Piano Sonatas                   | Ronald Brautigam                                            |
| Classical | Mozart, Wolfgang Amadeus | Serenades and Divertimenti      | Camerata Academica des Mozarteums Salzburg with Sandor Vegh |
| Classical | Mozart, Wolfgang Amadeus | String Quartets                 | Amadeus Quartet                                             |
| Classical | Mozart, Wolfgang Amadeus | String Quintets                 | Amadeus Quartet                                             |
| Classical | Mozart, Wolfgang Amadeus | Symphonies                      | Berlin Philharmonic with Karl Bohm                          |

Test set:
| Genre    | Composer  | Type of Work    | Recording                                    |
| -------- | --------- | --------------- | -------------------------------------------- |
| Romantic | Beethoven | Overtures       | Leipzig Gewandhaus Orchestra with Kart Masur |
| Romantic | Beethoven | Piano Concertos | Wilhelm Kempff                               |
| Romantic | Beethoven | Piano Sonatas   | Alfred Brendel                               |
| Romantic | Beethoven | String Quartets | Emerson String Quartet                       |
| Romantic | Beethoven | Symphonies      | Berlin Philharmonic with Herbert von Karajan |

I will evaluate the LLMs based on the following tags:
* Composer - last name goes first. Use wikipedia for reference
* Album
* Year Recorded - a four-digit year
* Orchestra - may be an orchestra, quartet, etc. translate the name of the ensemble into English
* Conductor - last name goes first
* Soloists - last name goes first. Separate multiple soloists with semi-colons
* Genre - allowed values are: Renaissance, Baroque, Classical, Romantic, 20th Century, 21st Century
* Work
* Work Number - should be padded to the length of the largest work number in the specific work
* InitialKey - single upper-case letter for major keys, upper-case letter plus minor for minor keys
* Catalog # - should be padded to the length of the largest catalog number in the composer's oeuvre
* Opus - should be padded to the length of the largest catalog number in the composer's oeuvre
* Opus Number - should be padded to the length of the largest catalog number in the composer's oeuvre
* Epithet
* Movement - use Roman numerals

